In [3]:
using CSV
using DataFrames
using Turing
using StatisticalRethinking
using StatsBase
using StatsPlots

# 5.1 Spurious association

Code 5.1

In [4]:
d = DataFrame(CSV.File("data/WaffleDivorce.csv"))
d[!,:D] = standardize(ZScoreTransform, d.Divorce)
d[!,:M] = standardize(ZScoreTransform, d.Marriage)
d[!,:A] = standardize(ZScoreTransform, d.MedianAgeMarriage);

Code 5.2

In [6]:
std(d.MedianAgeMarriage)

1.2436303013880823

Code 5.3

In [7]:
@model function model_m5_1(age, divorce)
    σ ~ Exponential(1)
    a ~ Normal(0, 0.2)
    bA ~ Normal(0, 0.5)
    μ = a .+ bA .* age
    divorce ~ MvNormal(μ, σ)
end

m5_1 = sample(model_m5_1(d.A, d.D), NUTS(), 1000)

┌ Info: Found initial step size
│   ϵ = 0.4
└ @ Turing.Inference /home/shmuma/.julia/packages/Turing/gwWVN/src/inference/hmc.jl:188
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00


Chains MCMC chain (1000×15×1 Array{Float64, 3}):

Iterations        = 501:1:1500
Number of chains  = 1
Samples per chain = 1000
Wall duration     = 10.19 seconds
Compute duration  = 10.19 seconds
parameters        = a, σ, bA
internals         = lp, n_steps, is_accept, acceptance_rate, log_density, hamiltonian_energy, hamiltonian_energy_error, max_hamiltonian_energy_error, tree_depth, numerical_error, step_size, nom_step_size

Summary Statistics
  parameters      mean       std   naive_se      mcse         ess      rhat    ⋯
      Symbol   Float64   Float64    Float64   Float64     Float64   Float64    ⋯

           σ    0.8247    0.0847     0.0027    0.0017   1467.4292    1.0004    ⋯
           a   -0.0062    0.1005     0.0032    0.0025   1230.7540    1.0010    ⋯
          bA   -0.5634    0.1140     0.0036    0.0045    843.7184    0.9992    ⋯
                                                                1 column omitted

Quantiles
  parameters      2.5%     25.0%     50.0%     75.0% 